In [7]:
import pandas as pd
import numpy as np
import os
import copy
import json
from tqdm.auto import tqdm

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
data_folder = '../data_Megogo/'
new_data_folder = '../generated_data/'
model_folder = new_data_folder +'models/factor_100/'
dic_folder = new_data_folder+'dic/'

In [9]:
import export_modules
from app.Recommender_implicit_new import Implicit
from app.metrics_new import Metrics

# Test

In [17]:
with open(dic_folder+'test_dic.json', 'r') as f:
    test_dic = json.load(f) 

# From 1 level

### Features

In [12]:
user_features_df = pd.read_csv(model_folder+'user_features.csv')
user_features_df['user_id'] = user_features_df['user_id'].apply(str)
user_features_df.head(2)

,uf0,uf1,uf2,uf3,uf4,uf5,uf6,uf7,uf8,uf9,...,uf91,uf92,uf93,uf94,uf95,uf96,uf97,uf98,uf99,user_id
0,-0.071293,0.559415,-0.072539,0.129628,-0.280565,0.059878,-0.283093,-0.060836,0.450761,-0.330126,...,0.242459,-0.129850,-0.201479,0.126578,-0.234366,0.084919,-0.537230,0.062307,0.434250,21603820
1,0.078816,-0.080879,0.154771,-0.018486,1.056839,0.269800,0.520433,0.593316,0.011278,-0.335845,...,0.323231,0.206529,0.454821,0.121880,-0.179581,-0.278110,-0.029591,0.093737,0.507237,35636970


In [16]:
new_col = ['user_id']

new_col.extend(list(user_features_df.columns[:-1]))

In [17]:
user_features_df.loc[:2, new_col]

,user_id,uf0,uf1,uf2,uf3,uf4,uf5,uf6,uf7,uf8,...,uf90,uf91,uf92,uf93,uf94,uf95,uf96,uf97,uf98,uf99
0,21603820,-0.071293,0.559415,-0.072539,0.129628,-0.280565,0.059878,-0.283093,-0.060836,0.450761,...,0.073832,0.242459,-0.129850,-0.201479,0.126578,-0.234366,0.084919,-0.537230,0.062307,0.434250
1,35636970,0.078816,-0.080879,0.154771,-0.018486,1.056839,0.269800,0.520433,0.593316,0.011278,...,-0.358628,0.323231,0.206529,0.454821,0.121880,-0.179581,-0.278110,-0.029591,0.093737,0.507237
2,78312976,-0.249220,-0.237675,0.078369,0.563539,-0.011536,0.416894,-0.021369,-0.241337,0.021629,...,0.130717,0.071178,0.311148,0.015022,0.053758,0.058183,0.319254,-0.365170,0.342792,0.276782


In [18]:
item_features_df = pd.read_csv(model_folder+'item_features.csv')
item_features_df['primary_video_id'] = item_features_df['primary_video_id'].apply(str)
item_features_df.head(2)

,if0,if1,if2,if3,if4,if5,if6,if7,if8,if9,...,if91,if92,if93,if94,if95,if96,if97,if98,if99,primary_video_id
0,-0.099971,-0.049025,0.043438,-0.019529,-0.075116,0.004447,-0.097199,0.012962,0.171468,-0.092855,...,-0.044163,0.090578,0.019817,-0.064349,0.073162,0.017504,-0.046425,-0.139511,-0.082321,9583642
1,0.019838,0.006003,-0.007013,-0.035202,0.112414,-0.047919,0.020665,0.049804,-0.024089,-0.021320,...,0.007876,0.018302,0.138939,-0.020511,0.099355,-0.107464,-0.008768,-0.038376,-0.028883,24645936


In [19]:
new_col = ['primary_video_id']

new_col.extend(list(item_features_df.columns[:-1]))

In [20]:
item_features_df.loc[:2, new_col]

,primary_video_id,if0,if1,if2,if3,if4,if5,if6,if7,if8,...,if90,if91,if92,if93,if94,if95,if96,if97,if98,if99
0,9583642,-0.099971,-0.049025,0.043438,-0.019529,-0.075116,0.004447,-0.097199,0.012962,0.171468,...,0.197261,-0.044163,0.090578,0.019817,-0.064349,0.073162,0.017504,-0.046425,-0.139511,-0.082321
1,24645936,0.019838,0.006003,-0.007013,-0.035202,0.112414,-0.047919,0.020665,0.049804,-0.024089,...,-0.022884,0.007876,0.018302,0.138939,-0.020511,0.099355,-0.107464,-0.008768,-0.038376,-0.028883
2,25397362,-0.097113,-0.008341,0.059431,0.133416,-0.030762,0.040883,0.019127,-0.072346,-0.015360,...,0.084288,-0.025701,-0.030473,0.066161,0.038716,-0.057714,0.005149,-0.012771,0.060515,-0.000277


### Predict

In [85]:
df_predict_10 = pd.read_csv(model_folder+'predict_10.csv')
df_predict_10[['user_id', 'primary_video_id']] = df_predict_10[['user_id', 'primary_video_id']].astype(str)
df_predict_10.head(2)

,user_id,primary_video_id
0,1088,2677761
1,1088,3696132


In [86]:
df_predict_20 = pd.read_csv(model_folder+'predict_20.csv')
df_predict_20[['user_id', 'primary_video_id']] = df_predict_20[['user_id', 'primary_video_id']].astype(str)
df_predict_20.head(2)

,user_id,primary_video_id
0,1088,2677761
1,1088,3696132


# Start 2 level

In [4]:
train = pd.read_csv(new_data_folder+'train_first_level.csv')
train[['user', 'item']] = train[['user', 'item']].astype(str)
train.columns = ['user_id', 'primary_video_id', 'watching_percentage']
train.head(2)

,user_id,primary_video_id,watching_percentage
0,21603820,9583642,0.0839
1,35636970,24645936,0.5990


In [5]:
train['is_relevant'] = train['watching_percentage'] >= 0.5
train['is_relevant'] = train['is_relevant'].apply(int)
train.head(2)

,user_id,primary_video_id,watching_percentage,is_relevant
0,21603820,9583642,0.0839,0
1,35636970,24645936,0.5990,1


In [6]:
train.is_relevant.value_counts()

1    5536587
0    2427810
Name: is_relevant, dtype: int64

In [7]:
train_good = train[train.is_relevant == 1]
train_bad = train[train.is_relevant == 0]

In [8]:
train_good = train_good.sort_values('watching_percentage', ascending=False)
train_bad = train_bad.sort_values('watching_percentage', ascending=True)

In [9]:
train_good = train_good.drop(['watching_percentage', 'is_relevant'], axis=1)
train_bad = train_bad.drop(['watching_percentage', 'is_relevant'], axis=1)

In [10]:
train_good = train_good.drop_duplicates()
train_bad = train_bad.drop_duplicates()

In [11]:
print(len(train_good))
print(len(train_bad))

3004447
1811982


In [12]:
user_items_good = train_good.groupby(['user_id'])['primary_video_id'].apply(lambda grp: list(grp)).to_dict()

In [13]:
user_items_bad = train_bad.groupby(['user_id'])['primary_video_id'].apply(lambda grp: list(grp)).to_dict()

In [33]:
with open(dic_folder+'user_items_good.json', 'w') as f:
    json.dump(user_items_good, f)
    
with open(dic_folder+'user_items_bad.json', 'w') as f:
    json.dump(user_items_bad, f)

In [38]:
del train_good
del train_bad

In [14]:
#user_items_good

In [15]:
#user_items_bad

In [16]:
users_in_good = set(list(user_items_good.keys()))
users_in_bad = set(list(user_items_bad.keys()))
users_in_both = users_in_good.intersection(users_in_bad)

In [18]:
users_in_test = list(test_dic.keys())

In [19]:
users_in_test_for_train = list(users_in_both.intersection(set(users_in_test)))

In [21]:
print(len(users_in_test))

187183


In [20]:
print(len(users_in_test_for_train))

153109


In [22]:
min_item = 10
k = 5

user_items_good_k = {}
user_items_bad_k = {}

users_for_train = []

for user in tqdm(users_in_test_for_train):
    good_items = user_items_good[user]
    bad_items = user_items_bad[user]
    
    if len(good_items) >= k and len(bad_items) >=min_item:
        users_for_train.append(user)
        user_items_good_k[user] = good_items[:k]
        user_items_bad_k[user] = bad_items[:k]

In [23]:
len(users_for_train)

36294

In [25]:
(len(users_for_train)*10.)/len(train)

0.04557030494587349

In [67]:
len(train)/(len(users_for_train)*10.)

7.176425482068841

In [68]:
len(users_for_train)

110980

In [44]:
#user_items_good_k

In [69]:
print(len(users_in_good))
print(len(users_in_bad))
print(len(users_in_both ))

390274
360368
348633


In [31]:
# for user, items in user_items_bad.items():
#     print(len(items))

In [32]:
# for user, items in user_items_good.items():
#     print(len(items))

In [26]:
def create_df_from_predict(dic_predict):
    user_res = []
    item_res = []
    for user, items in dic_predict.items():
        row_user = [user]*len(items)
        user_res.extend(row_user)
        item_res.extend(items)

    return pd.DataFrame({'user_id': user_res, 'primary_video_id': item_res})

In [30]:
df_good = create_df_from_predict(user_items_good_k)
df_good['is_relevant'] = 1
df_good.head(2)

,user_id,primary_video_id,is_relevant
0,40092468,31657320,1
1,40092468,18526881,1


In [29]:
df_bad = create_df_from_predict(user_items_bad_k)
df_bad['is_relevant'] = 0
df_bad.head(2)

,user_id,primary_video_id,is_relevant
0,40092468,11536584,0
1,40092468,18763922,0


In [31]:
df = df_good.append(df_bad)
df.index = range(len(df))
df.head(2)

,user_id,primary_video_id,is_relevant
0,40092468,31657320,1
1,40092468,18526881,1


In [32]:
df.to_csv(model_folder+'new_df_good_bad_5.csv', index=None)

# Upload meta info

In [21]:
meta = pd.read_csv(new_data_folder+ 'film_meta.csv')
meta['primary_video_id'] = meta['primary_video_id'].apply(str)
meta.head(2)

,primary_video_id,title,type,year,country,rating_imdb,rating_kinopoisk,score_by_popular,score_by_recommended,quality,...,Семейные,Союзмультфильм,Спортивные,Триллеры,Ужасы,Фантастика,Фильмы,Фильмы в 3D,Футбол,Юмористические
0,20069583,Мой лучший друг,FILM,2006,Франция,6.7,7.417,0.0,9283.0,FullHD,...,0,0,0,0,0,0,0,0,0,0
1,11492142,Бэндслэм,FILM,2009,США,6.3,6.766,16.0,8238.0,FullHD,...,0,0,0,0,0,0,0,0,0,0


In [26]:
#meta.columns

In [30]:
meta.iloc[:3,:12]

,primary_video_id,title,type,year,country,rating_imdb,rating_kinopoisk,score_by_popular,score_by_recommended,quality,age_limit,for_kids
0,20069583,Мой лучший друг,FILM,2006,Франция,6.7,7.417,0.0,9283.0,FullHD,16,0
1,11492142,Бэндслэм,FILM,2009,США,6.3,6.766,16.0,8238.0,FullHD,12,0
2,24885597,Толстяки,FILM,2009,Испания,6.7,6.962,9.0,6700.0,HD,16,0


# Merge

In [82]:
merg = df.merge(meta, how='inner', on='primary_video_id')
merg.head(2)

,user_id,primary_video_id,is_relevant,title,type,year,country,rating_imdb,rating_kinopoisk,score_by_popular,...,Семейные,Союзмультфильм,Спортивные,Триллеры,Ужасы,Фантастика,Фильмы,Фильмы в 3D,Футбол,Юмористические
0,23019489,31148166,1,Рэй и пожарный патруль,SERIAL,2016,Корея Южная,0.0,0.0,25.0,...,0,0,0,0,0,1,0,0,0,0
1,40670883,31148166,1,Рэй и пожарный патруль,SERIAL,2016,Корея Южная,0.0,0.0,25.0,...,0,0,0,0,0,1,0,0,0,0


In [87]:
merg = merg.merge(user_features_df, on='user_id', how='inner')
merg = merg.merge(item_features_df, on='primary_video_id', how='inner')

In [88]:
merg.head(2)

,user_id,primary_video_id,is_relevant,title,type,year,country,rating_imdb,rating_kinopoisk,score_by_popular,...,if165,if166,if167,if168,if169,if170,if171,if172,if173,if174
0,23019489,31148166,1,Рэй и пожарный патруль,SERIAL,2016,Корея Южная,0.0,0.0,25.0,...,0.024826,-0.015084,0.058009,0.005374,-0.060746,0.034847,0.022787,-0.047644,-0.019371,-0.053590
1,23019489,31148166,1,Рэй и пожарный патруль,SERIAL,2016,Корея Южная,0.0,0.0,25.0,...,0.001421,0.002654,0.004027,0.001517,0.001386,-0.001390,0.003549,0.005014,0.000207,0.000232


# Catboost

In [89]:
import catboost
from catboost import *
from catboost import CatBoostClassifier

In [91]:
target = merg['is_relevant']
merg = merg.drop(['user_id','primary_video_id', 'is_relevant'], axis=1)
merg.head(2)

,title,type,year,country,rating_imdb,rating_kinopoisk,score_by_popular,score_by_recommended,quality,age_limit,...,if165,if166,if167,if168,if169,if170,if171,if172,if173,if174
0,Рэй и пожарный патруль,SERIAL,2016,Корея Южная,0.0,0.0,25.0,10199.0,No information,0,...,0.024826,-0.015084,0.058009,0.005374,-0.060746,0.034847,0.022787,-0.047644,-0.019371,-0.053590
1,Рэй и пожарный патруль,SERIAL,2016,Корея Южная,0.0,0.0,25.0,10199.0,No information,0,...,0.001421,0.002654,0.004027,0.001517,0.001386,-0.001390,0.003549,0.005014,0.000207,0.000232


In [ ]:
cat_features = [0,1,2,3,8,9,10]

gen_ind = list(range(11,61)) #64
cat_features.extend(gen_ind)

In [ ]:
X_train, X_validation, y_train, y_validation = train_test_split(merg, target, train_size=0.5, random_state=4, shuffle=True)